In [ ]:
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, Optional
import re

def calculate_metrics(df: pd.DataFrame) -> Tuple[float, float, int, int, int]:
    """
    Calculate accuracy (excluding abstains), coverage, and abstain count.
    
    Returns:
        accuracy: float (accuracy excluding abstains)
        coverage: float (percentage of non-abstain predictions)
        correct: int (number of correct predictions)
        total_non_abstain: int (total non-abstain predictions)
        abstains: int (number of abstains)
    """
    # Normalize labels to lowercase for comparison
    df = df.copy()
    df['llm_label_lower'] = df['llm_label'].str.lower().str.strip()
    df['gold_label_lower'] = df['gold_label'].str.lower().str.strip()
    
    # Count abstains
    abstains = (df['llm_label_lower'] == 'abstain').sum()
    
    # Filter out abstains
    non_abstain = df[df['llm_label_lower'] != 'abstain']
    
    # Calculate metrics
    total_rows = len(df)
    total_non_abstain = len(non_abstain)
    
    if total_non_abstain > 0:
        correct = (non_abstain['llm_label_lower'] == non_abstain['gold_label_lower']).sum()
        accuracy = correct / total_non_abstain
    else:
        correct = 0
        accuracy = 0.0
    
    # Coverage: percentage of non-abstain predictions
    coverage = (total_non_abstain / total_rows * 100) if total_rows > 0 else 0.0
    
    return accuracy, coverage, correct, total_non_abstain, abstains

def parse_configuration(filename: str) -> Dict[str, str]:
    """
    Parse the filename to extract configuration details.
    
    Returns dict with:
        - prompts: 'Few-shot' or 'Zero-shot'
        - pipeline: 'Baseline', 'EM-RAG', or 'EM-KGRAG'
        - evidence_format: '-', 'Text', 'Python-code', or 'JSON'
        - hop_type: 'One-hop' or 'Multi-hop'
        - top_k: 'k=1' or 'k=3' (extracted from filename)
        - dataset: 'green_claim' or 'emerald_data'
        - config_key: unique key for matching (prompts_pipeline_evidence_hop_topk)
    """
    filename_lower = filename.lower()
    
    # Extract dataset type
    if filename_lower.startswith('green_claim_'):
        dataset = 'green_claim'
    elif filename_lower.startswith('emerald_data_'):
        dataset = 'emerald_data'
    else:
        dataset = 'unknown'
    
    # Extract prompts
    if 'few' in filename_lower:
        prompts = 'Few-shot'
    elif 'zero' in filename_lower:
        prompts = 'Zero-shot'
    else:
        prompts = 'Unknown'
    
    # Extract pipeline
    if 'baseline' in filename_lower:
        pipeline = 'Baseline'
    elif 'rag' in filename_lower:
        pipeline = 'EM-RAG'
    else:
        pipeline = 'EM-KGRAG'
    
    # Extract evidence format
    if 'baseline' in filename_lower:
        evidence_format = '-'
    elif 'rag' in filename_lower:
        evidence_format = 'Text'
    elif 'python' in filename_lower:
        evidence_format = 'Python-code'
    else:
        evidence_format = 'JSON'
    
    # Extract hop type
    if 'one_hop' in filename_lower:
        hop_type = 'One-hop'
    elif 'multi_hop' in filename_lower:
        hop_type = 'Multi-hop'
    else:
        hop_type = 'Unknown'
    
    # Extract top-k value (look for patterns like "_1_", "_3_", "hop_1", "hop_3")
    top_k = 'Unknown'
    # Try to find the k value after "hop_"
    match = re.search(r'hop_(\d+)', filename_lower)
    if match:
        k_value = match.group(1)
        top_k = f'k={k_value}'
    
    # Create a configuration key for matching
    config_key = f"{prompts}_{pipeline}_{evidence_format}_{hop_type}_{top_k}"
    
    return {
        'prompts': prompts,
        'pipeline': pipeline,
        'evidence_format': evidence_format,
        'hop_type': hop_type,
        'top_k': top_k,
        'dataset': dataset,
        'config_key': config_key
    }

def process_dataset(file_path: str) -> Optional[Dict]:
    """
    Process a single dataset and return its metrics and configuration.
    """
    try:
        df = pd.read_csv(file_path)
        accuracy, coverage, correct, total_non_abstain, abstains = calculate_metrics(df)
        
        config = parse_configuration(Path(file_path).name)
        
        return {
            'file': file_path,
            'accuracy': accuracy,
            'coverage': coverage,
            'abstains': abstains,
            'total_rows': len(df),
            **config,
            'success': True
        }
    except FileNotFoundError:
        print(f"Warning: {file_path} not found")
        return None
    except KeyError as e:
        print(f"Warning: Required column not found in {file_path}: {e}")
        return None
    except Exception as e:
        print(f"Warning: Error processing {file_path}: {e}")
        return None

def create_comparison_table(green_claim_files: list, emerald_data_files: list):
    """
    Create a comparison table showing green_claim and emerald_data datasets side-by-side.
    """
    # Process all files
    green_claim_results = {}
    emerald_data_results = {}
    results_dir = 'results/'
    print("\nProcessing files...")
    for file in green_claim_files:
        file = results_dir + file
        result = process_dataset(file)
        if result:
            green_claim_results[result['config_key']] = result
    
    for file in emerald_data_files:
        file = results_dir + file
        result = process_dataset(file)
        if result:
            emerald_data_results[result['config_key']] = result
    
    # Get all unique configurations
    all_configs = sorted(set(green_claim_results.keys()) | set(emerald_data_results.keys()))
    
    if not all_configs:
        print("No configurations found.")
        return
    
    # Print header
    print(f"\n{'='*180}")
    print("SIDE-BY-SIDE COMPARISON: green_claim vs emerald_data DATASETS")
    print(f"{'='*180}")
    
    # Column headers
    print(f"\n{'Prompts':<12s} {'Pipeline':<12s} {'Hop Type':<12s} {'Top-K':<8s} {'Evidence':<15s} "
          f"{'Acc_green_claim':<12s} {'Cov_green_claim':<12s} {'Abs_green_claim':<12s} "
          f"{'Acc_emerald_data':<12s} {'Cov_emerald_data':<12s} {'Abs_emerald_data':<12s}")
    print(f"{'-'*180}")
    
    # Print each configuration
    for config_key in all_configs:
        green_claim = green_claim_results.get(config_key)
        emerald_data = emerald_data_results.get(config_key)
        
        # Get configuration from whichever exists
        config = green_claim if green_claim else emerald_data
        if not config:
            continue
        
        prompts = config['prompts']
        pipeline = config['pipeline']
        hop_type = config['hop_type']
        top_k = config['top_k']
        evidence = config['evidence_format']
        
        # green_claim dataset metrics
        if green_claim:
            acc_green_claim = f"{green_claim['accuracy']*100:.2f}%"
            cov_green_claim = f"{green_claim['coverage']:.2f}%"
            abs_green_claim = f"{green_claim['abstains']}"
        else:
            acc_green_claim = "N/A"
            cov_green_claim = "N/A"
            abs_green_claim = "N/A"
        
        # emerald_data dataset metrics
        if emerald_data:
            acc_emerald_data = f"{emerald_data['accuracy']*100:.2f}%"
            cov_emerald_data = f"{emerald_data['coverage']:.2f}%"
            abs_emerald_data = f"{emerald_data['abstains']}"
        else:
            acc_emerald_data = "N/A"
            cov_emerald_data = "N/A"
            abs_emerald_data = "N/A"
        
        print(f"{prompts:<12s} {pipeline:<12s} {hop_type:<12s} {top_k:<8s} {evidence:<15s} "
              f"{acc_green_claim:<12s} {cov_green_claim:<12s} {abs_green_claim:<12s} "
              f"{acc_emerald_data:<12s} {cov_emerald_data:<12s} {abs_emerald_data:<12s}")
    
    # Print summary statistics
    print(f"\n{'-'*180}")
    print("SUMMARY STATISTICS")
    print(f"{'-'*180}")
    
    # Calculate averages for green_claim
    if green_claim_results:
        avg_acc_green_claim = sum(r['accuracy'] for r in green_claim_results.values()) / len(green_claim_results)
        avg_cov_green_claim = sum(r['coverage'] for r in green_claim_results.values()) / len(green_claim_results)
        total_abs_green_claim = sum(r['abstains'] for r in green_claim_results.values())
        
        print(f"{'green_claim - Average:':<40s} Accuracy: {avg_acc_green_claim*100:.2f}%  |  Coverage: {avg_cov_green_claim:.2f}%  |  Total Abstains: {total_abs_green_claim}")
    
    # Calculate averages for emerald_data
    if emerald_data_results:
        avg_acc_emerald_data = sum(r['accuracy'] for r in emerald_data_results.values()) / len(emerald_data_results)
        avg_cov_emerald_data = sum(r['coverage'] for r in emerald_data_results.values()) / len(emerald_data_results)
        total_abs_emerald_data = sum(r['abstains'] for r in emerald_data_results.values())
        
        print(f"{'emerald_data - Average:':<40s} Accuracy: {avg_acc_emerald_data*100:.2f}%  |  Coverage: {avg_cov_emerald_data:.2f}%  |  Total Abstains: {total_abs_emerald_data}")

def main():
    emerald_data_var=[
    "emerald_data_few_baseline.csv",
    "emerald_data_few_one_hop_3.csv",
    "emerald_data_few_rag.csv",
    "emerald_data_zero_baseline.csv",
    "emerald_data_zero_one_hop_3.csv",
    "emerald_data_zero_rag.csv",
    ]
    green_claim_var=[
    "green_claim_few_baseline.csv",
    "green_claim_few_one_hop_3.csv",
    "green_claim_few_rag.csv",
    "green_claim_zero_baseline.csv",
    "green_claim_zero_one_hop_3.csv",
    "green_claim_zero_rag.csv",
    ]
    
    print("\n" + "="*180)
    print("ACCURACY ANALYSIS: GREEN CLAIMS vs EMERALD DATA DATASETS")
    print("="*180)
    print("\nConfiguration Legend:")
    print("  - Prompts: Few-shot or Zero-shot")
    print("  - Pipeline: Baseline | EM-RAG | EM-KGRAG")
    print("  - Hop Type: One-hop | Multi-hop")
    print("  - Top-K: k=1 | k=3 (number of evidence paths retrieved)")
    print("  - Evidence Format: - (baseline) | Text (RAG) | Python-code | JSON")
    print("  - Accuracy: Excluding abstains")
    print("  - Coverage: Percentage of non-abstain predictions")
    
    create_comparison_table(green_claim_var, emerald_data_var)
    
    print("\n" + "="*180)
    print("ANALYSIS COMPLETE")
    print("="*180)

if __name__ == "__main__":
    main()


ACCURACY ANALYSIS: SMALL vs MIXED DATASETS

Configuration Legend:
  - Prompts: Few-shot or Zero-shot
  - Pipeline: Baseline | EM-RAG | EM-KGRAG
  - Hop Type: One-hop | Multi-hop
  - Top-K: k=1 | k=3 (number of evidence paths retrieved)
  - Evidence Format: - (baseline) | Text (RAG) | Python-code | JSON
  - Accuracy: Excluding abstains
  - Coverage: Percentage of non-abstain predictions

Processing files...

SIDE-BY-SIDE COMPARISON: SMALL vs MIXED DATASETS

Prompts      Pipeline     Hop Type     Top-K    Evidence        Acc_small    Cov_small    Abs_small    Acc_mixed    Cov_mixed    Abs_mixed   
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Few-shot     Baseline     Unknown      Unknown  -               100.00%      31.37%       35           N/A          N/A          N/A         
Few-shot     EM-KGRAG     Multi-hop    k=1      JSON            88.57%   